# Transformaciones
### Transformaciones de los archivos para usarlos en las funciones

Visualizamos los df que tenemos

In [42]:
import pandas as pd

f_Games = 'data//Steam-Games(clean).csv'
f_Items = 'data//Items(Transf).csv'
f_Reviews = 'data//Reviews(Transf).csv'

df_Games = pd.read_csv(f_Games)
df_Items = pd.read_csv(f_Items)
df_Reviews = pd.read_csv(f_Reviews)

print("User_Items")
display(df_Items.head(0))
print("Games")
display(df_Games.head(0))
print("Reviews")
display(df_Reviews.head(0))

User_Items


,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks


Games


,id,app_name,title,publisher,genres,url,release_date,tags,reviews_url,specs,price,early_access,developer,year


Reviews


,user_id,funny,last_edited,item_id,helpful,recommend,review,posted


## 1. **def PlayTimeGenre( genero : str )**:
Debe devolver año con mas horas jugadas para dicho género.
- Ejemplo de retorno: {“Año de lanzamiento con más horas jugadas para Género X” : 2013}

Considerando la funciónn 1 y la 2 debo de hacer un df con user_id, item_id, playtime_forever, genres

In [43]:
# Dividimos la columna 'genres' en múltiples registros
#Convierto en lista genres

total = len(df_Games)
print("tamaño del df original",total)
df_Games['genres'] = df_Games['genres'].str.split(',')
df_Games = df_Games.explode('genres')

# Luego, seleccionamos las columnas que necesitas
ndf_Games = df_Games[['id', 'title', 'genres', 'release_date']]

ntotal = len(ndf_Games)
print("Tamaño del nuevo df: ",ntotal)
ndf_Games.head(10)

tamaño del df original 29781
Tamaño del nuevo df:  71935


,id,title,genres,release_date
0,761140.0,Lost Summoner Kitty,['Action',2018-01-04
0,761140.0,Lost Summoner Kitty,'Casual',2018-01-04
0,761140.0,Lost Summoner Kitty,'Indie',2018-01-04
0,761140.0,Lost Summoner Kitty,'Simulation',2018-01-04
0,761140.0,Lost Summoner Kitty,'Strategy'],2018-01-04
1,643980.0,Ironbound,['Free to Play',2018-01-04
1,643980.0,Ironbound,'Indie',2018-01-04
1,643980.0,Ironbound,'RPG',2018-01-04
1,643980.0,Ironbound,'Strategy'],2018-01-04
2,670290.0,Real Pool 3D - Poolians,['Casual',2017-07-24


In [44]:
#union de ndf_Games y df_Items
# Realiza una fusión (merge) de los DataFrames utilizando las columnas 'item_id' y 'id'
merged_df = df_Items.merge(ndf_Games, left_on='item_id', right_on='id', how='inner')

total = len(merged_df)
print("Tamaño del merged df: ",total)
display(merged_df.head(0))
display(merged_df[['id','item_id','item_name','title','genres',"user_id",'release_date']].head(15))

Tamaño del merged df:  9938488


,user_id,items_count,steam_id,user_url,item_id,item_name,playtime_forever,playtime_2weeks,id,title,genres,release_date


,id,item_id,item_name,title,genres,user_id,release_date
0,10.0,10,Counter-Strike,Counter-Strike,['Action'],76561197970982479,2000-11-01
1,10.0,10,Counter-Strike,Counter-Strike,['Action'],js41637,2000-11-01
2,10.0,10,Counter-Strike,Counter-Strike,['Action'],Riot-Punch,2000-11-01
3,10.0,10,Counter-Strike,Counter-Strike,['Action'],doctr,2000-11-01
4,10.0,10,Counter-Strike,Counter-Strike,['Action'],corrupted_soul,2000-11-01
5,10.0,10,Counter-Strike,Counter-Strike,['Action'],cadmusthreepointoh,2000-11-01
6,10.0,10,Counter-Strike,Counter-Strike,['Action'],76561198089393905,2000-11-01
7,10.0,10,Counter-Strike,Counter-Strike,['Action'],WeiEDKrSat,2000-11-01
8,10.0,10,Counter-Strike,Counter-Strike,['Action'],death-hunter,2000-11-01
9,10.0,10,Counter-Strike,Counter-Strike,['Action'],ABSOLUTLY_NOTHING,2000-11-01


In [45]:
# Extraer el año de la columna release_date y agregarlo como 'year'
merged_df['year'] = pd.to_datetime(merged_df['release_date']).dt.year

# Agrupar por 'genres', 'year' y sumar la columna 'playtime_forever'
grouped_df = merged_df.groupby(['genres', 'year']).agg({'playtime_forever': 'sum'}).reset_index()

# Renombrar la columna resultante a 'hrs_jugadas'
grouped_df = grouped_df.rename(columns={'playtime_forever': 'hrs_jugadas'})

# Acomodo las columnas
grouped_df = grouped_df[['year', 'genres', 'hrs_jugadas']]

# Ordeno el DataFrame por 'year' y 'genres'
grouped_df = grouped_df.sort_values(by=['year', 'genres'])

total = len(grouped_df)
print("Tamaño del df: ",total)
display(grouped_df.head(4))

Tamaño del df:  848


,year,genres,hrs_jugadas
2,1983,'Adventure',3582
68,1983,'Casual'],3582
421,1983,['Action',3582
3,1984,'Adventure',384


Limpio el campo de genre de espacios y caracteres diferentes a letras y convierto a minusculas

In [46]:
def clean(cadena):
    # Elimina espacios y caracteres que no sean letras y luego convierte a minúsculas
    cadena_limpia = ''.join(caracter.lower() for caracter in cadena if caracter.isalpha())
    return cadena_limpia

# Aplica la función a la columna 'genres'
grouped_df['genres'] = grouped_df['genres'].apply(clean)

total = len(grouped_df)
print("Numero de registros: ",total)
grouped_df.head()

Numero de registros:  848


,year,genres,hrs_jugadas
2,1983,adventure,3582
68,1983,casual,3582
421,1983,action,3582
3,1984,adventure,384
69,1984,casual,384


#### Este df es que se va a usar para la funcion 1 : genre_playtime.csv

In [47]:
# Guardo el df
total = len(grouped_df)
print("Tamaño del df: ",total)
grouped_df.to_csv('data//genre_playtime.csv', index=False) 

Tamaño del df:  848


# 2.- def UserForGenre( genero : str )
Debe devolver el usuario que acumula más horas jugadas para el género dado y una lista de la acumulación de horas jugadas por año.

- Ejemplo de retorno: {“Usuario con más horas jugadas para Género X” : us213ndjss09sdf, “Horas jugadas”:[{Año: 2013, Horas: 203}, {Año: 2012, Horas: 100}, {Año: 2011, Horas: 23}]}

In [48]:
# Dividimos la columna 'genres' en múltiples registros
#Convierto en lista genres

total = len(df_Games)
print("tamaño del df original",total)
df_Games['genres'] = df_Games['genres'].str.split(',')
df_Games = df_Games.explode('genres')

# Seleccionamos las columnas necesarias
ndf1_Games = df_Games[['id', 'title','genres', 'release_date']].copy()

# Extraer el año de la columna release_date y agregarlo como 'year'
ndf1_Games['year'] = pd.to_datetime(ndf_Games['release_date']).dt.year
display(ndf1_Games.head(0))

#Elimino la columna release_date
ndf1_Games = ndf1_Games.drop(columns=['release_date'])

ntotal = len(ndf1_Games)
print("Tamaño del nuevo df: ",ntotal)
#ndf1_Games.head(10)

tamaño del df original 71935


,id,title,genres,release_date,year


Tamaño del nuevo df:  71935


In [49]:
#union de ndf_Games y df_Items
# Realiza una fusión (merge) de los DataFrames utilizando las columnas 'item_id' y 'id'
merged_df = df_Items.merge(ndf1_Games, left_on='item_id', right_on='id', how='inner')

ndf = merged_df.drop(columns=['items_count','item_id','item_name','steam_id','user_url','playtime_2weeks','id','title'])

# Agrupar por 'user_id','genres' y 'year' y sumar la columna 'playtime_forever'
ndf = ndf.groupby(['user_id','genres', 'year']).agg({'playtime_forever': 'sum'}).reset_index()

# Renombrar la columna resultante a 'min_jugados'
ndf = ndf.rename(columns={'playtime_forever': 'min_jugados'})

# Acomodo las columnas
ndf = ndf[['user_id', 'genres','year', 'min_jugados']]

# Agrupa por 'user_id' y suma los minutos jugados
ndf['total_min'] = ndf.groupby('user_id')['min_jugados'].transform('sum')

# Ordeno el DataFrame por minutos jugados 
ndf = ndf.sort_values(by=['total_min', 'user_id', 'year', 'genres'], ascending=[False, True, True, True])


total = len(ndf)
print("Tamaño del df: ",total)
display(ndf.head(2))

Tamaño del df:  4724899


,user_id,genres,year,min_jugados,total_min
3098898,REBAS_AS_F-T,'Free to Play'],1996,0,9565845
3099046,REBAS_AS_F-T,['Action',1996,0,9565845


In [50]:
# Crea un nuevo DataFrame que contiene solo las filas con 'user_id' igual a 'REBAS_AS_F-T'
df_jugador_masminutos = ndf[(ndf['user_id'] == 'REBAS_AS_F-T') & (ndf['min_jugados'] != 0)]
#Ordeno el df
df_final = df_jugador_masminutos.sort_values(['year','genres','min_jugados'],ascending=[True,True,False])


total = len(df_final)
print("Tamaño del df: ",total)

Tamaño del df:  288


Paso a minuscula los usuarios y los generos

In [54]:
def clean(cadena):
    # Elimina espacios y caracteres que no sean letras y luego convierte a minúsculas
    cadena_limpia = ''.join(caracter.lower() for caracter in cadena if caracter.isalpha())
    return cadena_limpia

# Aplica la función a la columna 'genres' en df_final
df_final['genres'] = df_final['genres'].apply(clean)

# Convierte a minúsculas la columna 'user_id' en df_final
df_final['user_id'] = df_final['user_id'].astype(str)
df_final['user_id'] = df_final['user_id'].str.lower()

total = len(df_final)
print("Número de registros: ", total)
df_final.head()

Número de registros:  288


,user_id,genres,year,min_jugados,total_min
3099066,rebas_as_f-t,action,1998,2,9565845
3099024,rebas_as_f-t,strategy,1999,1,9565845
3099067,rebas_as_f-t,action,1999,225,9565845
3099150,rebas_as_f-t,rpg,1999,1,9565845
3098884,rebas_as_f-t,freetoplay,2001,11,9565845


### Guardo el df del Jugador con mas minutos jugados

In [55]:
#Jugador con mas minutos jugados 
# Guardo el df
total = len(df_final)
print("Tamaño del df: ",total)
df_final.to_csv('data//jugador_masminutos.csv', index=False) 

Tamaño del df:  288


# 3. def UsersRecommend( año : int ): 
Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos/neutrales)
- Ejemplo de retorno: [{“Puesto 1” : X}, {“Puesto 2” : Y},{“Puesto 3” : Z}]
    

In [56]:
display(df_Reviews.head(0))
display(df_Games.head(0))

,user_id,funny,last_edited,item_id,helpful,recommend,review,posted


,id,app_name,title,publisher,genres,url,release_date,tags,reviews_url,specs,price,early_access,developer,year


In [57]:
# Fuciono (merge) de los DataFrames en base al campo común 'item_id' e 'id'
df_combined = df_Reviews.merge(df_Games[['id', 'title']], left_on='item_id', right_on='id', how='left')
# Limpia id
df_combined = df_combined.drop('id', axis=1)

# Agregar una columna 'year' con el año de la columna 'posted'
df_combined['year'] = pd.to_datetime(df_combined['posted']).dt.year

# Sumar los valores verdaderos de la columna 'recommend' por 'year' y 'user_id'
df_summarized = df_combined.groupby(['item_id','title','year'])['recommend'].sum().reset_index()

# Renombrar la columna 'recommend' a 'total_review'
df_summarized.rename(columns={'recommend': 'total_review'}, inplace=True)

# Ordenar el DataFrame por 'total_review'
df_summarized = df_summarized.sort_values(by=['total_review','year'], ascending=[False,True])

df_summarized.head(3)

,item_id,title,year,total_review
78,440,Team Fortress 2,2014,2786
3429,304930,Unturned,2014,2575
2721,252490,Rust,2014,2232


In [58]:
# Ordena el DataFrame por año (year) y luego por total_review en orden descendente
df_sorted = df_summarized.sort_values(by=['year', 'total_review'], ascending=[True, False])

# Crea DataFrames separados para el top y el bottom
top_reviews_df = pd.DataFrame()
bottom_reviews_df = pd.DataFrame()

# Itera a través de los años únicos en el DataFrame
for year in df_sorted['year'].unique():
    # Obtén las filas correspondientes a un año específico
    df_year = df_sorted[df_sorted['year'] == year]
    
    # Obtén los 3 juegos con más revisiones
    top_reviews = df_year.head(3)
    
    # Obtén los 3 juegos con menos revisiones
    bottom_reviews = df_year.tail(3)
    
    # Concatena los resultados en los DataFrames respectivos
    top_reviews_df = pd.concat([top_reviews_df, top_reviews])
    bottom_reviews_df = pd.concat([bottom_reviews_df, bottom_reviews])

# Muestra los DataFrames resultantes
print("Top Reviews:")
display(top_reviews_df)

print("\nBottom Reviews:")
display(bottom_reviews_df)

Top Reviews:


,item_id,title,year,total_review
74,440,Team Fortress 2,2010,20
108,1250,Killing Floor,2010,6
213,4000,Garry's Mod,2010,6
75,440,Team Fortress 2,2011,150
1423,105600,Terraria,2011,92
91,620,Portal 2,2011,52
76,440,Team Fortress 2,2012,488
1424,105600,Terraria,2012,164
1508,200210,Realm of the Mad God,2012,85
77,440,Team Fortress 2,2013,1398



Bottom Reviews:


,item_id,title,year,total_review
971,40100,Supreme Commander 2,2010,1
988,40800,Super Meat Boy,2010,1
1219,62100,Chime,2010,1
1355,97100,Section 8®: Prejudice™,2011,1
623,18700,And Yet It Moves,2011,0
1236,63940,Men of War: Vietnam,2011,0
1037,42920,The Kings' Crusade,2012,0
1200,57740,Jagged Alliance - Back in Action,2012,0
1752,209100,Resident Evil: Operation Raccoon City,2012,0
2248,232050,Eador. Masters of the Broken World,2013,0


In [59]:
# Cambio a minusculas title
top_reviews_df['title'] = top_reviews_df['title'].str.lower()
top_reviews_df.head(4)

,item_id,title,year,total_review
74,440,team fortress 2,2010,20
108,1250,killing floor,2010,6
213,4000,garry's mod,2010,6
75,440,team fortress 2,2011,150


In [60]:
# Top de juegos mas recomendados 
# Guardo el df
total = len(top_reviews_df)
print("Tamaño del df: ",total)
top_reviews_df.to_csv('data//top_reviews3.csv', index=False) 

Tamaño del df:  18


# 4. def UsersNotRecommend( año : int ): 
Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado. (reviews.recommend = False y comentarios negativos)
- Ejemplo de retorno: [{“Puesto 1” : X}, {“Puesto 2” : Y},{“Puesto 3” : Z}]
    

In [3]:
# Fuciono (merge) de los DataFrames en base al campo común 'item_id' e 'id'
df_combined = df_Reviews.merge(df_Games[['id', 'title']], left_on='item_id', right_on='id', how='left')
# Limpia id
df_combined = df_combined.drop('id', axis=1)
# Agregar una columna 'year' con el año de la columna 'posted'
df_combined['year'] = pd.to_datetime(df_combined['posted']).dt.year

# Filtra las filas donde 'recommend' es False
df_false_recommend = df_combined[df_combined['recommend'] == False]
# Agrupa y suma los valores falsos por 'year'
df_summarized = df_false_recommend.groupby(['year','item_id','title'])['recommend'].count().reset_index()
df_summarized.rename(columns={'recommend': 'false_recommend_count'}, inplace=True)

total = len(df_summarized)
print("Total de registros en el df: ",total)
df_summarized.sort_values(['year','false_recommend_count'],ascending=[True,False])

Total de registros en el df:  1359


,year,item_id,title,false_recommend_count
0,2010,440,Team Fortress 2,1
2,2011,440,Team Fortress 2,2
3,2011,18700,And Yet It Moves,2
1,2011,240,Counter-Strike: Source,1
4,2011,33460,From Dust,1
...,...,...,...,...
1353,2015,412400,GASP,1
1354,2015,416670,Gunslingers,1
1356,2015,419070,Warhammer: End Times - Vermintide Sigmar's Ble...,1
1357,2015,421770,Pool Nation FX - Unlock Online,1


In [4]:
# Ordena el DataFrame por 'year' y 'false_recommend_count' en orden descendente
df_sorted = df_summarized.sort_values(by=['year', 'false_recommend_count'], ascending=[True, False])

# Crea un DataFrame vacío para almacenar los resultados
top_false_recommendations = pd.DataFrame()

# Itera a través de los años únicos en el DataFrame
for year in df_sorted['year'].unique():
    # Filtra las filas correspondientes a un año específico
    df_year = df_sorted[df_sorted['year'] == year]
    
    # Obtén los 3 juegos con los mayores valores de false_recommend_count
    top_games = df_year.head(3)
    
    # Concatena los resultados en el DataFrame final
    top_false_recommendations = pd.concat([top_false_recommendations, top_games])

# Muestra el DataFrame final
top_false_recommendations.sort_values(['year','false_recommend_count'],ascending=[True,False])

,year,item_id,title,false_recommend_count
0,2010,440,Team Fortress 2,1
2,2011,440,Team Fortress 2,2
3,2011,18700,And Yet It Moves,2
1,2011,240,Counter-Strike: Source,1
6,2012,440,Team Fortress 2,1
7,2012,10680,Aliens vs. Predator™,1
8,2012,24240,PAYDAY™ The Heist,1
79,2013,209160,Call of Duty®: Ghosts,14
17,2013,440,Team Fortress 2,12
101,2013,224540,Ace of Spades: Battle Builder,10


In [5]:
# Top de juegos mas recomendados 
# Guardo el df
total = len(top_false_recommendations)
print("Tamaño del df: ",total)
top_false_recommendations.to_csv('data//top_reviews4.csv', index=False) 

Tamaño del df:  16


### Caso II
En este caso se considera todos los 1ros lugares y todos los 2dos lugares y todos los 3eros lugares por año

In [55]:
# Ordena el DataFrame por 'year' y 'false_recommend_count' en orden descendente
df_sorted = df_summarized.sort_values(by=['year', 'false_recommend_count'], ascending=[True, False])

# Crea un DataFrame vacío para almacenar los resultados
top_false_recommendations = pd.DataFrame()

# Itera a través de los años únicos en el DataFrame
for year in df_sorted['year'].unique():
    # Filtra las filas correspondientes a un año específico
    df_year = df_sorted[df_sorted['year'] == year]
    
    # Encuentra los valores únicos de 'false_recommend_count' en el año actual
    unique_counts = df_year['false_recommend_count'].unique()
    
    # Itera a través de los valores únicos y agrega los juegos correspondientes
    for count in unique_counts:
        top_games = df_year[df_year['false_recommend_count'] == count]
        top_false_recommendations = pd.concat([top_false_recommendations, top_games])

# Muestra el DataFrame final
top_false_recommendations

,year,item_id,title,false_recommend_count
0,2010,440,Team Fortress 2,1
2,2011,440,Team Fortress 2,2
3,2011,18700,And Yet It Moves,2
1,2011,240,Counter-Strike: Source,1
4,2011,33460,From Dust,1
...,...,...,...,...
1353,2015,412400,GASP,1
1354,2015,416670,Gunslingers,1
1356,2015,419070,Warhammer: End Times - Vermintide Sigmar's Ble...,1
1357,2015,421770,Pool Nation FX - Unlock Online,1


## 5. def sentiment_analysis( año : int )
Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.
- Ejemplo de retorno: {Negative = 182, Neutral = 120, Positive = 278}

In [1]:
import pandas as pd

f_Reviews = 'data//Reviews(Transf).csv'
df_Reviews = pd.read_csv(f_Reviews)

display(df_Reviews.head(0))
total = len(df_Reviews)
print("Cantidad de registros: ",total)

,user_id,funny,last_edited,item_id,helpful,recommend,review,posted


Cantidad de registros:  45944


Selecciono las columnas 

In [2]:
reviews_text = df_Reviews[['review','posted']].copy()
reviews_text.head()

# Convierte la columna "posted" a tipo de dato datetime
reviews_text['posted'] = pd.to_datetime(reviews_text['posted'])

# Crea la nueva columna "year_posted" que contiene el año de "posted"
reviews_text['year'] = reviews_text['posted'].dt.year
reviews_text.drop(columns=['posted'],inplace=True)

reviews_text.head(2)

,review,year
0,Simple yet with great replayability. In my opi...,2011
1,It's unique and worth a playthrough.,2011


## Traduciendo los comentarios
***********************************

In [3]:
from mtranslate import translate
import sys
import time

# -------------------------------------------------------------------------
#-----------------------------     Traductor 
def traducir_texto(texto, idioma_destino):
    try:
        traduccion = translate(texto, idioma_destino)
        return traduccion
    except Exception as e:
        # print(f"Error al traducir: {str(e)}")
        return "SIN TRADUCCIÓN"
# -------------------------------------------------------------------------  
# -----------------  Contador
def contador(ciclo, multiplo):
    if ciclo % multiplo == 0:
        sys.stdout.write("\r{}".format(ciclo))
        sys.stdout.flush()    

In [4]:
from langdetect import detect

df = reviews_text.copy()

# Crear una nueva columna 'is_english' con valor predeterminado False (0)
df['traduccion'] = 0

# Iterar a través de las filas y detectar el idioma de cada comentario
for index, row in df.iterrows():
    contador(index,20)
    review = row['review']
    try:
        lang = detect(review)
        if lang == 'en':
            df.at[index, 'traduccion'] = 1
    except:
        # Si no se puede detectar el idioma, mantener el valor predeterminado (False)
        pass


0

45940

In [5]:
df_SinTraducir = df[df['traduccion'] == False].copy()
df_SinTraducir.reset_index(drop=True, inplace=True)
df_Traducidos =  df[df['traduccion'] == True].copy()
df_Traducidos.reset_index(drop=True, inplace=True)

total = len(df_SinTraducir)
print("Tamaño: ",total)
df_SinTraducir.head(2)

Tamaño:  9318


,review,year,traduccion
0,Git gud,2014,0
1,É muito bom,2014,0


In [6]:
# Crear una lista para almacenar los comentarios traducidos
comentarios_traducidos = []

# Iterar a través de las filas y traducir los comentarios
for i, row in df_SinTraducir.iterrows():
    review = row['review']
    traduccion = traducir_texto(review, 'en')
    comentarios_traducidos.append(traduccion)
    contador(i, 100)

# Agregar la lista de comentarios traducidos al DataFrame
df_SinTraducir['comentario'] = comentarios_traducidos

# Resto del código para guardar lotes si es necesario


9300

In [19]:
df_ya_traducido = df_SinTraducir.copy()
# Eliminar la columna 'review'
df_ya_traducido = df_ya_traducido.drop(['review','traduccion'], axis=1)
# renombro la columna comentario
df_ya_traducido.rename(columns={'comentario': 'review'}, inplace=True)

df = df_ya_traducido.copy()
df.head(3)

,year,review
0,2014,Give god
1,2014,It is very good
2,2013,es la rajaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...


In [26]:
#Para hacer el analisis de los archivos traducidos
df = df_Traducidos.copy()
total = len(df_Traducidos)
print("Total de registros: ",total)


Total de registros:  36626


### Analisis de sentimiento

Este analisis de sentimiento lo hice en partes por que estaba tardando mucho y no tenia el tiempo de esperar a que terminara el analisis, asi que llame por partes el df y lo fuí guardando para poder hacerlo en varias sesiones

In [27]:
from textblob import TextBlob

# Tamaño de lote (1000 registros)
batch_size = 5000


# Inicializa el contador
contador_registros = 0

# Bucle para procesar el DataFrame en lotes
for i in range(0, len(df), batch_size):
    batch = df[i:i+batch_size].copy()  # Obtén el lote actual

    # Realiza el análisis de sentimiento solo para reseñas en inglés
    batch['Sentimiento'] = batch.apply(lambda row: TextBlob(row['review']).sentiment.polarity, axis=1)
    # Guarda el lote procesado
    batch.to_csv(f'bloquesData/lote-({i}-{i+batch_size}).csv', index=False)

    # Incrementa el contador
    contador_registros += len(batch)
    print(f'Registros guardados: {contador_registros}/{len(df)}')

print('Procesamiento completado')
df.head(3)


Registros guardados: 5000/36626
Registros guardados: 10000/36626
Registros guardados: 15000/36626
Registros guardados: 20000/36626
Registros guardados: 25000/36626
Registros guardados: 30000/36626
Registros guardados: 35000/36626
Registros guardados: 36626/36626
Procesamiento completado


,review,year,traduccion
0,Simple yet with great replayability. In my opi...,2011,1
1,It's unique and worth a playthrough.,2011,1
2,Great atmosphere. The gunplay can be a bit chu...,2011,1


Cargo de nuevo todos los archivos batch que se crearon y los convierto en un solo df y los vuelvo a guardar 

In [28]:
import pandas as pd
import os

# Directorio donde se guardaron los archivos CSV por bloques
directorio = 'bloquesData/'

# Lista para almacenar los DataFrames de bloques
dataframes = []

# Itera a través de los archivos CSV en el directorio
for filename in os.listdir(directorio):
    if filename.endswith(".csv"):
        # Lee cada archivo CSV y lo agrega a la lista de DataFrames
        filepath = os.path.join(directorio, filename)
        df = pd.read_csv(filepath)
        dataframes.append(df)

# Combina los DataFrames en uno solo
df_combinado = pd.concat(dataframes, ignore_index=True)

# Opcional: Guarda el DataFrame combinado en un archivo CSV
df_combinado.to_csv(f'{directorio}AnalisisSentimeinto.csv', index=False)


In [34]:
import pandas as pd

f_analisisSentimiento = 'bloquesData//AnalisisSentimeinto.csv'
df = pd.read_csv(f_analisisSentimiento)

display(df.head(0))
total = len(df)
print("Cantidad de registros: ",total)

,year,review,Sentimiento,traduccion


Cantidad de registros:  45944


In [36]:
#Para hacer el analisis de los archivos traducidos
total = len(df)
print("Total de registros: ",total)

df = df.drop(['review','traduccion'],axis=1)
df.head()

Total de registros:  45944


,year,Sentimiento
0,2014,0.00
1,2014,0.91
2,2013,0.00
3,2014,0.15
4,2015,1.00


In [39]:
# Agrupa por año y calcula la suma de los sentimientos para cada categoría
resultado = df.groupby('year')['Sentimiento'].agg([('Positivo', lambda x: (x > 0).sum()),
                                                    ('Negativo', lambda x: (x < 0).sum()),
                                                    ('Neutral', lambda x: (x == 0).sum())]).reset_index()

# Si deseas llenar los valores NaN con 0, puedes hacerlo de la siguiente manera
resultado.fillna(0, inplace=True)

resultado



,year,Positivo,Negativo,Neutral
0,2010,47,11,8
1,2011,361,93,57
2,2012,756,205,163
3,2013,4216,1287,792
4,2014,13007,4688,2931
5,2015,9965,4414,2943


In [40]:
# Guardo el df
resultado.to_csv('data//SetimientoxAño.csv', index=False) 

In [ ]:
from textblob import TextBlob

# Función para aplicar el análisis de sentimientos en el idioma correcto
def analyze_sentiment(text, language):
    try:
        if language != 'unknown':
            blob = TextBlob(text)
            blob = blob.translate(to=language)  # Traduce al idioma detectado
            return blob.sentiment.polarity
        else:
            return 0.0  # Valor predeterminado para reseñas sin idioma
    except:
        return 0.0


In [11]:
# Tamaño de lote (1000 registros)
batch_size = 1000
cont = 0

# Precesando el df por partes
for i in range(0, len(reviews_text), batch_size):
    batch = reviews_text[i:i+batch_size]  # Obtén el lote actual
    
    # Realiza el análisis de sentimiento para el lote actual
    # batch['Sentimiento'] = batch['review'].apply(lambda x: TextBlob(x).sentiment.polarity)
    batch['sentiment'] = batch.apply(lambda row: analyze_sentiment(row['review'], row['language']), axis=1)
    # Guarda el lote procesado
    batch.to_csv(f'lote_{i}-{i+batch_size}.csv', index=False) 
    # Incrementa el contador
    cont+= len(batch)
    
    print(f'Registros guardados: {cont}/{len(reviews_text)}')

print('Procesamiento completado')

C:\Users\adeal\AppData\Local\Temp\ipykernel_5932\3896061260.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['sentiment'] = batch.apply(lambda row: analyze_sentiment(row['review'], row['language']), axis=1)


Registros guardados: 1000/48498


C:\Users\adeal\AppData\Local\Temp\ipykernel_5932\3896061260.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['sentiment'] = batch.apply(lambda row: analyze_sentiment(row['review'], row['language']), axis=1)


Registros guardados: 2000/48498


C:\Users\adeal\AppData\Local\Temp\ipykernel_5932\3896061260.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batch['sentiment'] = batch.apply(lambda row: analyze_sentiment(row['review'], row['language']), axis=1)


Registros guardados: 3000/48498


In [ ]:
# Aplicar el análisis de sentimientos en el idioma correcto
reviews_text['sentiment'] = reviews_text.apply(lambda row: analyze_sentiment(row['review'], row['language']), axis=1)

# Clasificar las reseñas en positivas, negativas y neutrales
reviews_text['sentiment_category'] = reviews_text['sentiment'].apply(lambda x: 'positivo' if x > 0 else ('negativo' if x < 0 else 'neutral'))


In [27]:
import numpy as np
import pandas as pd
import gdown

# Descargar el archivo desde Google Drive
file_id = '1-Eve6MwFsy1VyhsQB7gTRItzd1Rb8V4M'
url = f'https://drive.google.com/uc?id={file_id}'
Users = 'Users1.csv'
gdown.download(url, Users, quiet=False)

def userdata(user_id):

    df = pd.read_csv(Users) 
    # Filtrar el DataFrame para obtener el registro con el valor deseado en user_id
    valor = df[df['user_id'] == user_id]
    
    if not valor.empty:
        # Obtener los valores de la columna Total Invertido y Porcentaje Reviews
        monto = valor['Total Invertido'].values[0]
        porcentaje = valor['Porcentaje Reviews'].values[0]
        return (monto, porcentaje)
    else:
        return (np.nan, np.nan)


# ---------------------------------------------------------#
usuario = '-2SV-vuLB-Kg'
monto, porcentaje = userdata(usuario)

if np.isnan(monto) and np.isnan(porcentaje):
    print(f"No se encontró ningún registro con user_id igual a {usuario}")
else:
    porcentaje = "{:.2f}%".format(porcentaje)
    print("Cantidad de dinero invertido:", monto)
    print("Porcentaje de recomendaciones: ", porcentaje)



Downloading...
From: https://drive.google.com/uc?id=1-Eve6MwFsy1VyhsQB7gTRItzd1Rb8V4M
To: c:\Users\adeal\Nextcloud\00-CodigoTK\BotCamps\Henry-Proyect1-Games1.1\Users1.csv
100%|██████████| 6.07M/6.07M [00:00<00:00, 9.52MB/s]


Cantidad de dinero invertido: 437.49
Porcentaje de recomendaciones:  7.35%


In [34]:
import pandas as pd

def countreviews(fe_ini, fe_fin):
    f_Reviews = 'Datos//Reviews-Recortado.csv'
    df = pd.read_csv(f_Reviews)
    display(df.head(2))

    # Definir el rango de fechas
    fecha_inicio = pd.to_datetime(fe_ini)
    fecha_final= pd.to_datetime(fe_fin)
   
    #verifica que sea un rango valido
    if fecha_inicio > fecha_final:
        print("El rango de fechas no es correcto.")
        del df
        return(-1)
    
    # Convierte las columnas de fecha a tipo datetime si aún no lo están
    df['posted'] = pd.to_datetime(df['posted'])
    
    # Filtra el DataFrame para incluir solo las filas dentro del rango de fechas y con 'recommend' igual a True
    filtered_df = df[(df['posted'] >= fecha_inicio) & (df['posted'] <= fecha_final) & df['recommend']]
    
    # Obtiene la cantidad de user_id únicos en el DataFrame filtrado
    unique_user_count = len(filtered_df['user_id'].unique())
    
    del df
    return unique_user_count



# ---------------------------------------------------------#
# Llamar a la función para contar las recomendaciones dentro del rango de fechas
recomendaciones_en_rango = countreviews('2013-06-15', '2014-03-10')

print(f"Cantidad de usuarios que recomendaron dentro del rango de fechas: {recomendaciones_en_rango}")


,item_id,user_id,recommend,posted
0,1250,76561197970982479,True,2011-11-05
1,22200,76561197970982479,True,2011-07-15


fechas ok
Cantidad de usuarios que recomendaron dentro del rango de fechas: 7026


## 3. def **genre( *`género` : str* )**:<br>
    Devuelve el `puesto` en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [49]:
import pandas as pd
import ast

f_Games = 'Datos//Games.csv'
f_Users_Items = 'Datos//Users_Items.csv'

df_Games = pd.read_csv(f_Games)
df_Users_Items = pd.read_csv(f_Users_Items)

###############################################
# Recorto los df para hacer pruebas
#df_Games = df_Games[:5000]
#df_Users_Items = df_Users_Items[:5000]
##############################################

display(df_Games.head(1))
display(df_Users_Items.head(1))

# Seleccionar solo algunas columnas de los df
df_Games = df_Games[['genres', 'item_id']]
df_Users_Items = df_Users_Items[['item_id', 'user_id', 'playtime_forever']]

# Fusionar los DataFrames en función de la columna 'item_id'
merged_df = pd.merge(df_Users_Items, df_Games, on='item_id', how='inner')

# Convertir las cadenas en la columna 'genres' en listas
merged_df['genres'] = merged_df['genres'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

merged_df.head(5)

,genres,app_name,title,release_date,price,item_id,developer
0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,Lost Summoner Kitty,2018-01-04,4.99,761140.0,Kotoshiro


,item_id,user_id,playtime_forever,price,Free
0,10,76561197970982479,6,9.99,NaN


,item_id,user_id,playtime_forever,genres
0,10,76561197970982479,6,[Action]
1,10,js41637,0,[Action]
2,10,Riot-Punch,0,[Action]
3,10,doctr,93,[Action]
4,10,corrupted_soul,108,[Action]


In [50]:
# Crear una lista para almacenar los diccionarios de cada fila
data = []

for index, row in merged_df.iterrows():
    genres = row['genres']
    if isinstance(genres, list):  # Verificar si 'genres' es una lista
        for gen in genres:
            #print(type(gen)," : ",gen)
            # Crear un diccionario con las columnas existentes y la nueva columna 'genero'
            new_row = {
                'item_id': str(row['item_id']),  # Convertir 'item_id' en una cadena (string)
                'user_id': row['user_id'],
                'playtime_forever': row['playtime_forever'],
                'genero': gen.lower()  # Agregar la nueva columna 'genero'
            }
            data.append(new_row)

# Crear un DataFrame a partir de la lista de diccionarios
df_playtime_forever = pd.DataFrame(data)

Elimino los registros con 0 o menos en la columna playtime_forever

In [55]:
# Contar la cantidad de registros antes de la eliminación
total_registros = len(df_playtime_forever)

# Filtrar los registros donde playtime_forever > 0
df_playtime_forever = df_playtime_forever[df_playtime_forever['playtime_forever'] > 0]

# Contar la cantidad de registros después de la eliminación
registros_eliminados = total_registros - len(df_playtime_forever)

# Imprimir la cantidad de registros eliminados y la cantidad que no se eliminaron
print(f"Registros eliminados: {registros_eliminados}")
print(f"Registros que no se eliminaron: {len(df_playtime_forever)}")


Registros eliminados: 0
Registros que no se eliminaron: 6928925


In [56]:
# Guardar el DataFrame en un archivo CSV
df_playtime_forever.to_csv('Datos//playtime_forever.csv', index=False) 

display(df_playtime_forever.head(1))

,item_id,user_id,playtime_forever,genero
0,10,76561197970982479,6,action


In [81]:
# Agrupa por el género y calcula la suma de 'Games-PlayTimeForEver'
df_genero_sum = df_playtime_forever.groupby('genero')['playtime_forever'].sum().reset_index()
# Ordeno el df de mayor a menor
df_genero_sum = df_genero_sum.sort_values(by='playtime_forever', ascending=False)
# Agrego el numero para el orden de mayor a menor
df_genero_sum['Posicion'] = range(1, len(df_genero_sum) + 1)
# Aplicar str.lower() a la columna 'genero' en df_genero_sum
df_genero_sum['genero'] = df_genero_sum['genero'].str.lower()

In [82]:
# Guardar el DataFrame en un archivo CSV
df_genero_sum.to_csv('Datos//Genero_Posicion.csv', index=False) 

df_genero_sum.head(1)

,genero,playtime_forever,Posicion
0,action,3113562606,1


In [29]:
#Devuelve la posicion del genero de acuerdo al numero de horas jugadas
def genre(genero):
    
    # Cargo el archivo al df
    f_genero = 'Datos//Genero_Posicion.csv'
    dataframe = pd.read_csv(f_genero)
    
    genero_a_buscar = genero.lower()
    # Buscar el género en el DataFrame
    resultado = dataframe[dataframe['genero'] == genero_a_buscar]

    if not resultado.empty:
        # Si se encontró el género, obtener su posición
        posicion = resultado.iloc[0]['Posicion']
        del dataframe
        return f'{genero_a_buscar} esta en el ranking {posicion} de generos en horas jugadas'
    else:
        # Si el género no se encontró, devolver un mensaje de error
        del dataframe
        return f"Género '{genero_a_buscar}' no encontrado."


##########################################################################
# Llamar a la función para encontrar la posición de un género específico
genero_buscado = 'free to play'
posicion = genre(genero_buscado)

print(posicion)

free to play esta en el ranking 7 de generos en horas jugadas


In [18]:
type(posicion)

str

## 4.- def **userforgenre( *`género` : str* )**:<br>
    `Top 5` de usuarios con más horas de juego en el género dado, con su URL (del user) y user_id.

Usamos playtime_forever.csv creado en el apartado 3 anterior

In [52]:
# Cargo el archivo al df
f_playtime_forever = 'Datos//playtime_forever.csv'
df_playtime_forever = pd.read_csv(f_playtime_forever)
display(df_playtime_forever.head(1))

,item_id,user_id,playtime_forever,genero
0,10,76561197970982479,6,action


In [67]:
import json

def userforgenre(genero):
    # Cargar solo las columnas necesarias
    f_playtime_forever = 'Datos//playtime_forever.csv'
    df_playtime_forever = pd.read_csv(f_playtime_forever, usecols=['user_id', 'genero', 'playtime_forever'])
    
    # Filtrar por género
    df_genero = df_playtime_forever[df_playtime_forever['genero'] == genero]
    
    if df_genero.empty:
        return f"Este género ({genero}) tiene 0 horas jugadas."
    
    # Agrupar por 'user_id' y calcular la suma de 'playtime_forever'
    resumen_genero = df_genero.groupby('user_id')['playtime_forever'].sum()
    
    # Tomar los 5 primeros usuarios con más horas de juego
    top_5_usuarios = resumen_genero.nlargest(5).reset_index()
    
    # Agregar una columna 'Lugar' para representar el ranking
    top_5_usuarios['Lugar'] = top_5_usuarios.index + 1
    
    # Reorganizar las columnas para que estén en el orden deseado
    top_5_usuarios = top_5_usuarios[['Lugar', 'user_id', 'playtime_forever']]
    
    # Eliminar DataFrames que ya no se necesitan para liberar memoria
    del df_playtime_forever, df_genero, resumen_genero

    #display(top_5_usuarios)
    json_data = top_5_usuarios.to_json(orient='records')
    data = json.loads(json_data)
    pretty_json = json.dumps(data, indent=4)
    #display(json_data)
    return pretty_json


###################################
# Llamar a la función
genero = 'free to play' 
#genero = 'RPG'
top_usuarios = userforgenre(genero)

# Imprimir los 5 usuarios con más horas de juego en el género especificado
print(top_usuarios)

[
    {
        "Lugar": 1,
        "user_id": "idonothack",
        "playtime_forever": 808241
    },
    {
        "Lugar": 2,
        "user_id": "shinomegami",
        "playtime_forever": 788553
    },
    {
        "Lugar": 3,
        "user_id": "Evilutional",
        "playtime_forever": 757973
    },
    {
        "Lugar": 4,
        "user_id": "tobscene",
        "playtime_forever": 498268
    },
    {
        "Lugar": 5,
        "user_id": "jwatt1991",
        "playtime_forever": 469357
    }
]


In [101]:
# Ordenar por 'user_id', 'genero' y 'playtime_forever' (ascendente)
df_ordenado = df_playtime_forever.sort_values(by=['genero','user_id', 'playtime_forever'], ascending=[True, False, False])

In [98]:
# Guardar el DataFrame en un archivo CSV
df_user_sum.to_csv('Datos//User_Posicion.csv', index=False) 

df_user_sum.head(1)

,user_id,playtime_forever,Posicion
50198,rebas_as_f-t,9590897,1


## 5.-def **developer( *`desarrollador` : str* )**: <br>
    `Cantidad` de items y `porcentaje` de contenido Free por año según empresa desarrolladora. <br>
Ejemplo de salida:<br>
    

**Activision** 
|   Año   |     Items      | Contenido Free|
|--------|-----------------|---------------|
|   2023  |        xx         |       27%      |
|   2022  |        xx         |       25%      |
|   xxxx  |        xx         |       xx%      |


In [68]:
f_Games = 'Datos//Games.csv'

# Especifica las columnas que deseas cargar
columnas = ['item_id', 'release_date', 'price', 'developer']
df_Games = pd.read_csv(f_Games, usecols=columnas)

df_Games.head(1)

,release_date,price,item_id,developer
0,2018-01-04,4.99,761140.0,Kotoshiro


In [101]:
# Convertir las columnas price y developer string minúsculas
df_Games['price'] = df_Games['price'].astype(str).str.lower()
df_Games['developer'] = df_Games['developer'].astype(str).str.lower()

# Eliminar filas duplicadas en función de la columna 'item_id'
df_Games.drop_duplicates(subset='item_id', keep='first', inplace=True)

# Convertir la columna 'release_date' a objetos de fecha
df_Games['release_date'] = pd.to_datetime(df_Games['release_date'], errors='coerce')

# Extraer el año de la columna 'release_date'
df_Games['release_year'] = df_Games['release_date'].dt.year

# Filtrar las filas con fechas válidas
df_Games = df_Games[df_Games['release_year'].notna()]

# Agrupar por 'developer', 'release_year' y calcular la suma de 'item_id' y la suma de 'price' igual a "free to play"
agregaciones = {
    'item_id': 'sum',  # Suma de 'item_id' por 'developer' y 'release_year'
    'price': lambda x: (x == 'free to play').sum()  # Cantidad de 'price' igual a "free to play" por 'developer' y 'release_year'
}

# Realizar la agrupación y agregar las columnas al DataFrame
df_resumen = df_Games.groupby(['developer', 'release_year']).agg(agregaciones).reset_index()

# Renombrar las columnas resultantes
df_resumen.rename(columns={'item_id': 'juegos_creados', 'price': 'free_to_play'}, inplace=True)

# Calcular la columna 'porcentaje'
df_resumen['porcentaje'] = (df_resumen['free_to_play'] / df_resumen['juegos_creados']) * 100

# Filtrar desarrolladores con juegos creados mayores a 0
df_resumen = df_resumen[df_resumen['juegos_creados'] > 0]

# Ordenar el DataFrame por 'developer', 'release_year' y 'porcentaje' en orden descendente
df_resumen = df_resumen.sort_values(by=['developer', 'release_year', 'porcentaje'], ascending=[True, False, False])

# Mostrar el resultado
print("Resumen por 'developer', 'release_year' ordenado por 'porcentaje' en orden descendente:")

Resumen por 'developer', 'release_year' ordenado por 'porcentaje' en orden descendente:


In [102]:
df_resumen

,developer,release_year,juegos_creados,free_to_play,porcentaje
0,+7 software,2016,459830.0,0,0.000000
1,"+mpact games, llc.",2017,349510.0,0,0.000000
2,.ez games,2017,647050.0,0,0.000000
3,.m.y.w.,2016,553830.0,0,0.000000
6,07th expansion,2017,1767310.0,0,0.000000
...,...,...,...,...,...
14782,萌石游戏,2017,687650.0,0,0.000000
14783,高考恋爱委员会,2015,396960.0,1,0.000252
14784,"高考恋爱委员会,days",2015,378400.0,0,0.000000
14786,"高考恋爱委员会,橘子班",2017,747900.0,0,0.000000


In [88]:
# Guardar el DataFrame en un archivo CSV
df_resumen.to_csv('Datos//freetoplay.csv', index=False) 

display(df_resumen.head(1))

,developer,release_year,juegos_creados,free_to_play,porcentaje
0,+7 software,2016,459830.0,0,0.0


In [104]:
def developer(desarrollador):

    df_freetoplay = pd.read_csv('Datos//freetoplay.csv')
    # Filtrar el DataFrame por el desarrollador especificado
    df_desarrollador = df_freetoplay[df_freetoplay['developer'] == desarrollador.lower()]
    
    # Seleccionar las columnas 'release_year' y 'porcentaje'
    df_resultado = df_desarrollador[['release_year','juegos_creados','free_to_play', 'porcentaje']].copy()
    
    # Renombrar las columnas
    df_resultado.rename(columns={'release_year': 'año','juegos_creados':'Total de juegos','free_to_play' : 'Total free', 'porcentaje': '% free'}, inplace=True)
    
    # Restablecer el índice del DataFrame resultante
    df_resultado.reset_index(drop=True, inplace=True)
    
    return df_resultado

desarrollador = "07th Expansion"
df = developer(desarrollador)
print("Desarrollador: ",desarrollador )
df

Desarrollador:  07th Expansion


,año,Total de juegos,Total free,% free
0,2015,721250.0,0,0.0
1,2016,1405910.0,0,0.0
2,2017,1767310.0,0,0.0


Funcion para el APIFast

In [96]:
def obtener_porcentaje_contenido(desarrollador):
    df_freetoplay = pd.read_csv('Datos//freetoplay.csv')
    
    # Filtrar el DataFrame por el desarrollador especificado
    df_desarrollador = df_freetoplay[df_freetoplay['developer'] == desarrollador.lower()]
    
    # Si no se encontraron datos para el desarrollador, devuelve None
    if df_desarrollador.empty:
        return None
    
    # Seleccionar las columnas 'release_year' y 'porcentaje'
    df_resultado = df_desarrollador[['release_year', 'porcentaje']].copy()
    
    # Renombrar las columnas
    df_resultado.rename(columns={'release_year': 'año', 'porcentaje': 'contenido_free'}, inplace=True)
    
    # Restablecer el índice del DataFrame resultante
    df_resultado.reset_index(drop=True, inplace=True)
    
    # Convertir el DataFrame a un diccionario de Python
    resultado_dict = df_resultado.to_dict(orient="records")
    
    return resultado_dict

@app.get("/developer/{desarrollador}")
async def developer(desarrollador: str, request: Request):
    # Obtener los datos del desarrollador
    data = obtener_porcentaje_contenido(desarrollador)
    
    if data is None:
        # Si no se encontraron datos, devuelve un mensaje de error
        message = f"No se encontraron datos del {desarrollador}"
        return templates.TemplateResponse("error.html", {"request": request, "message": message})

    # Si se encontraron datos, devuelve la plantilla con los datos
    return templates.TemplateResponse("tabla_porcentajes.html", {"request": request, "data": data, "desarrollador": desarrollador})


In [97]:
desarrollador = "1C Company"
res = developer(desarrollador)
print(res)

[{'año': 2004, '% contenido_free': 0.0}, {'año': 2007, '% contenido_free': 0.0}, {'año': 2008, '% contenido_free': 0.0}, {'año': 2009, '% contenido_free': 0.0}, {'año': 2010, '% contenido_free': 0.0}, {'año': 2011, '% contenido_free': 0.0}]


## 6.- def **sentiment_analysis( *`año` : int* )**: 
Según el año de lanzamiento, se devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento.<br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ejemplo de retorno:&nbsp;**{Negative = 182, Neutral = 120, Positive = 278}**

In [6]:
import pandas as pd
from textblob import TextBlob

f_Reviews = 'Datos//Reviews(Transf).csv'

# Especifica las columnas que deseas cargar
columnas = ['posted', 'recommend', 'review']
df_Reviews = pd.read_csv(f_Reviews, usecols=columnas)

# Obtener el número de registros (filas)
numero_de_registros = df_Reviews.shape[0]
print('numero_de_registros', numero_de_registros)

df_Reviews.head(1)

numero_de_registros 49186


,recommend,review,posted
0,True,Simple yet with great replayability. In my opi...,2011-11-05


In [7]:
# Eliminar registros con valores nulos
df_Reviews.dropna(subset=['review'], inplace=True)
# Convierte a fecha
df_Reviews['posted'] = pd.to_datetime(df_Reviews['posted'])
# Extraer el año
df_Reviews['year'] = df_Reviews['posted'].dt.year
df_Reviews = df_Reviews.drop('posted',axis=1)

# verificar 'reviews' sea una cadena (str)
df_Reviews['review'] = df_Reviews['review'].astype(str)
# Convertir a minúsculas
df_Reviews['review'] = df_Reviews['review'].str.lower()
df_Reviews['review'] = df_Reviews['review'].str.strip()

# Obtener el número de registros (filas)
numero_de_registros = df_Reviews.shape[0]
print('numero_de_registros', numero_de_registros)
df_Reviews.head(1)

numero_de_registros 49159


,recommend,review,year
0,True,simple yet with great replayability. in my opi...,2011


In [8]:
df = df_Reviews.copy()

def analizar_sentimiento(texto):
    analysis = TextBlob(texto)
    if analysis.sentiment.polarity > 0:
        return 'Positiva'
    elif analysis.sentiment.polarity == 0:
        return 'Neutral'
    else:
        return 'Negativa'

# Filtra el DataFrame para eliminar filas con valores nulos en la columna 'transcript'
df_Reviews = df_Reviews.dropna(subset=['review'])

reviews = df_Reviews['review']
# Aplicar la función de análisis de sentimiento a cada revisión y crear una nueva columna en el DataFrame
df_Reviews.loc[:, 'sentimiento'] = df_Reviews['review'].apply(analizar_sentimiento)


In [12]:
df_Reviews.head(1)

,recommend,review,year,sentimiento
0,True,simple yet with great replayability. in my opi...,2011,Positiva


In [14]:
# Calcula la suma de sentimientos por categoría ('Positiva', 'Negativa', 'Neutral') y agrupa por año.
sentimientos_agregados = df_Reviews.groupby('year')['sentimiento'].value_counts().unstack().fillna(0)

# Calcula la suma de registros por año.
registros_por_año = df_Reviews.groupby('year')['review'].count().reset_index()

# Renombra las columnas de los sentimientos.
sentimientos_agregados.columns = ['Negativa', 'Neutral', 'Positiva']

# Combina los datos en un nuevo DataFrame.
df_resultado = registros_por_año.merge(sentimientos_agregados, on='year')

# Ordena las columnas según tu especificación.
df_resultado = df_resultado[['year', 'review', 'Positiva', 'Negativa', 'Neutral']]

# Imprime el nuevo DataFrame.
display(df_resultado)


,year,review,Positiva,Negativa,Neutral
0,2010,66,47,11,8
1,2011,532,363,97,72
2,2012,1215,770,217,228
3,2013,6786,4116,1275,1395
4,2014,22098,12717,4551,4830
5,2015,18462,9846,4292,4324


In [41]:
# Guardar el DataFrame en un archivo CSV
df_resultado.to_csv('Datos//SentimientoxAño.csv', index=False) 

display(df_resultado.head(1))

NameError: name 'df_resultado' is not defined

In [23]:
def sentimiento(año):

    f_Sentimiento = 'Datos//SentimientoxAño.csv'
    df_Sentimiento = pd.read_csv(f_Sentimiento)

    # Filtra el DataFrame para el año especificado.
    año_filtrado = df_Sentimiento[df_Sentimiento['year'] == año]
    
    # Verifica si el año especificado existe en el DataFrame.
    if not año_filtrado.empty:
        # Obtiene los valores de las columnas 'Negativa', 'Neutral', 'Positiva' y 'review' para el año especificado.
        Negativa = año_filtrado['Negativa'].values[0]
        Neutral = año_filtrado['Neutral'].values[0]
        Positiva = año_filtrado['Positiva'].values[0]
        reviews = año_filtrado['review'].values[0]

        # Crea y retorna el string deseado.
        resultado = f'Reviews del año {año} : Negativos = {Negativa}, Neutrales = {Neutral}, Positivos = {Positiva} || Total: {reviews}'
        return resultado
    else:
        return f'No hay datos disponibles para el año {año}'


año = 2011  # Reemplaza esto con el año que desees consultar.
informacion = obtener_informacion_por_año(año)
print(informacion)


Reviews del año 2011 : Negativos = 97, Neutrales = 72, Positivos = 363 || Total: 532
